# Import Libraries

In [18]:
# Import Libaries
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Functions to combine data

In [2]:
# Function takes in csv file path and returns a list of new column names to be added (excluding "planning area")
def return_new_col_list(suffix,filepath):
    # Instantiate new list that will take in the update list with suffix
    col_orig_list = []
    new_col_list = []  
    combined_list = []
    col_list = pd.read_csv(filepath, nrows=0).columns.tolist()    
    for col in col_list:
        if(col != "planning_area"):
            col_w_suffix = suffix + "_"+col
            new_col_list.append(col_w_suffix)
            col_orig_list.append(col)
    combined_list.append(new_col_list)
    combined_list.append(col_orig_list)    
    return combined_list

In [3]:
# Function to append new column to dictionary and to assign value as np.NaN
def add_new_col_df(new_col_list, dataframe):
    dataframe = dataframe.copy() # removes warning error of editing data frame slice.
    for col in new_col_list:
        dataframe[col] = None
    return  dataframe

In [4]:
# Function to take in string, splice by space to update word Token to be upper case
def format_upper_lower_case(text):
    text_list = text.split()
    combine_format_text = ""
    # loop through text
    for text in text_list:
        text_length = len(text)
        formatted_text = ""
        # format first charcater in text to upper case, else lower case
        for i in range(0,text_length):
            if(i==0):
                formatted_text += text[i].upper()
            else:
                formatted_text += text[i].lower()
        # combine text with formated indivdual text. add space if it is second text                
        if(len(combine_format_text) == 0):
            combine_format_text += formatted_text
        else:
            formatted_text = " " + formatted_text
            combine_format_text +=  formatted_text
    return combine_format_text   

# Check that the format text function works.    
format_upper_lower_case('ANG MO KIO')

'Ang Mo Kio'

In [5]:
# Function to create dictionary file with planning_area as reference and returns dictionary ~~ to create when cfm dictionary idea works
def convert_df_to_dict(orig_col_list, dataframe):
    # initialisted dictionary
    temp_dict = {}
    for index, row in dataframe.iterrows():
        retrieve_planning_area = row["planning_area"]
        # creates empty nested dictionary where key is planning area
        temp_dict[retrieve_planning_area] = {}
        for orig_col in orig_col_list:
            retrieve_val = row[orig_col]
            temp_dict[retrieve_planning_area][orig_col] = retrieve_val
    return temp_dict

In [6]:
# Creates a function that takes combine existing resale flat data, creates new column and maps data
def join_resale_flat_new_data(unique_resale_planning_area,new_col_list,orig_col_list,resale_df,new_df):
    temp_new_df_dict = convert_df_to_dict(orig_col_list, new_df)
    for planning_area in unique_resale_planning_area:

        # initialised col list counter
        col_list_count = 0
        for new_col in new_col_list:
            # retrieve original column based on counter e.g. new col is p1_employee while original column is employee 
            # to get data  from dictionary
            orig_col = orig_col_list[col_list_count]

            # convert text to upper and lower key using function
            fmt_planning_area = format_upper_lower_case(planning_area) 

            retrieve_dict_value = temp_new_df_dict[fmt_planning_area][orig_col]


            col_list_count += 1
            resale_df.loc[(resale_df['planning_area']==planning_area)
                                                    ,new_col] = retrieve_dict_value
    return resale_df

In [12]:
# Function to loop through the list of new_df with predermined suffix to call functions too append data to list
def add_all_dataset(new_df_filepath_suffix, resale_df):
    # Get unique "planning_query_town" values
    resale_df = resale_df.copy()
    unique_resale_planning_area = resale_long_lat_fmt_only_concat_town["planning_area"].unique().tolist()   

    
    for filepath_suffix in new_df_filepath_suffix:
        retrieve_new_df_filepath = filepath_suffix[0]
        retrieve_new_df_suffix = filepath_suffix[1]
        # Get original list and newly named column list
        # newly named columns will be used as new columns in combined resale flat data. original column will be used to
        # reference the existing dataframe.

        # create list of new column names to be added
        combined_col_list = return_new_col_list(retrieve_new_df_suffix,retrieve_new_df_filepath)

        # Get list of newly_named_columns
        new_col_list = combined_col_list[0]

        # Get list of original named columns
        orig_col_list = combined_col_list[1]
        
        # Retrieve dataframe base on filepath
        new_df = pd.read_csv(retrieve_new_df_filepath)
        
        
        resale_df = join_resale_flat_new_data(unique_resale_planning_area
                          , new_col_list
                          ,orig_col_list,resale_df
                          ,new_df)
    return resale_df

# Combine Resale Flat Data with One Map Cleaned Dataset
- One Map data set has been cleaned and average.
- The data will be combined with Resale Flat data

## Read in Resale Flat Information

In [8]:
# Read Dataset
resale_long_lat_fmt_only_concat_town = pd.read_csv('../assets/data_clean/resale_w_plantown_clean.csv')

# Dataset Info
resale_long_lat_fmt_only_concat_town.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903393 entries, 0 to 903392
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   town                 903393 non-null  object 
 1   flat_type            903393 non-null  object 
 2   block                903393 non-null  object 
 3   street_name          903393 non-null  object 
 4   storey_range         903393 non-null  object 
 5   floor_area_sqm       903393 non-null  float64
 6   flat_model           903393 non-null  object 
 7   lease_commence_date  903393 non-null  int64  
 8   resale_price         903393 non-null  float64
 9   remaining_lease      903393 non-null  int64  
 10  address              903393 non-null  object 
 11  longitude            903393 non-null  float64
 12  latitude             903393 non-null  float64
 13  month_fmt            903393 non-null  object 
 14  month                903393 non-null  int64  
 15  year             

- Observed that the data set has been read in for Resale flat price

In [9]:
# Rename planning_query_town to planning_area for resale_flat df
resale_long_lat_fmt_only_concat_town.rename(columns={"planning_query_town":"planning_area"}, inplace=True)

## Review data from One Map to be added to Resale Flat data

- Dataset from One Map that will not be added to Resale Flat Data:
    - Planning Area Id (planning_area_id_cleaned.csv) will not be added to the Resale Flat Data as planning area id is not required to be reference for other one map content. 


- Dataset from One Map that will be added to Resale Flat Data (filepath and suffix). Suffix will be used in the new columns to be added for Resale Flat data.
    1. Econmic Status: '../assets/data_clean/population_economic_status_cleaned.csv', 'P1'
    2. Education Status: '../assets/data_clean/population_education_status_cleaned.csv', 'P2'
    3. Ethnic Status: '../assets/data_clean/population_ethnic_status_cleaned.csv', 'P3'
    4. Work Income For Household (Monthly): '../assets/data_clean/population_household_monthly_income_status_cleaned.csv', 'P4'
    5. Population household structure data: '../assets/data_clean/population_household_structure_cleaned.csv', 'P5'
    6. Income From Work Data: '../assets/data_clean/population_income_from_work_cleaned.csv', 'P6'
    7. Planning Area Industry: '../assets/data_clean/population_planning_area_industry_cleaned.csv', 'P7'
    8. Population Language Literacy: '../assets/data_clean/population_language_literacy_cleaned.csv', 'P8'
    9. Population Martial Status: '../assets/data_clean/population_martial_status_cleaned.csv', 'P9'
    10. Population Transport Mode To School: '../assets/data_clean/population_transport_mode_school_cleaned.csv', 'P10'
    11. Population Transport Mode To Work: '../assets/data_clean/population_transport_mode_work_cleaned.csv', 'P11'
    12. Population Age Group: '../assets/data_clean/population_age_group_cleaned.csv', 'P12'
    13. Population Religion: '../assets/data_clean/population_religion_cleaned.csv', 'P13'
    14. Population Spoken Language: '../assets/data_clean/population_spoken_language_cleaned.csv', 'P14'
    15. Population Tenancy: '../assets/data_clean/population_tenancy_cleaned.csv', 'P15'
    16. Population Dwelling Type Household Data: '../assets/data_clean/population_dwelling_type_household_cleaned.csv', 'P16'
    17. Population Dwelling Type Population: '../assets/data_clean/population_dwelling_type_population_cleaned.csv', 'P17'

In [10]:
# Instantiate list of file path and suffix
new_df_filepath_suffix = [['../assets/data_clean/population_economic_status_cleaned.csv','P1']
                          , ['../assets/data_clean/population_education_status_cleaned.csv','P2']        
                          , ['../assets/data_clean/population_ethnic_status_cleaned.csv', 'P3']  
                          , ['../assets/data_clean/population_household_monthly_income_status_cleaned.csv', 'P4'] 
                          , ['../assets/data_clean/population_household_structure_cleaned.csv', 'P5'] 
                          
                          , ['../assets/data_clean/population_income_from_work_cleaned.csv', 'P6'] 
                          , ['../assets/data_clean/population_planning_area_industry_cleaned.csv', 'P7'] 
                          , ['../assets/data_clean/population_language_literacy_cleaned.csv', 'P8'] 
                          , ['../assets/data_clean/population_martial_status_cleaned.csv', 'P9'] 
                          , ['../assets/data_clean/population_transport_mode_school_cleaned.csv', 'P10'] 
                          
                          , ['../assets/data_clean/population_transport_mode_work_cleaned.csv', 'P11'] 
                          , ['../assets/data_clean/population_age_group_cleaned.csv', 'P12'] 
                          , ['../assets/data_clean/population_religion_cleaned.csv', 'P13'] 
                          , ['../assets/data_clean/population_spoken_language_cleaned.csv', 'P14'] 
                          , ['../assets/data_clean/population_tenancy_cleaned.csv', 'P15'] 
                          
                          , ['../assets/data_clean/population_dwelling_type_household_cleaned.csv', 'P16'] 
                          , ['../assets/data_clean/population_dwelling_type_population_cleaned.csv', 'P17'] 
                        ]

In [13]:
# Add new column to resale flat data
combined_resale_df = add_all_dataset(new_df_filepath_suffix,resale_long_lat_fmt_only_concat_town)

In [21]:
# view data
combined_resale_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903393 entries, 0 to 903392
Data columns (total 240 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    town                                   object 
 1    flat_type                              object 
 2    block                                  object 
 3    street_name                            object 
 4    storey_range                           object 
 5    floor_area_sqm                         float64
 6    flat_model                             object 
 7    lease_commence_date                    int64  
 8    resale_price                           float64
 9    remaining_lease                        int64  
 10   address                                object 
 11   longitude                              float64
 12   latitude                               float64
 13   month_fmt                              object 
 14   month                             

In [17]:
# export coordinateList to csv
combined_resale_df.to_csv('../assets/data_clean/resale_combine_clean.csv', encoding='utf-8', index=False)

# export as pickle file
combined_resale_df.to_pickle('../assets/data_clean/resale_combine_clean.pkl')

- Observed that data has been exported